In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# --- 1. Valores de Base ---
ctd_base = 217062.50
margem_venda_base_perc = 0.30 # 30%
ganho_bruto_cliente_anual_base = 20493000.00
percentual_bonus_consultoria = 0.05 # 5%
teto_bonus_anual_consultoria = 1000000.00
bonus_custo_gestao_anual = 10000.00 # Custo fixo da consultoria para gerir o bônus

# --- 2. Definições das Variações ---
variacoes_ctd_perc_sobre_base = [0, 0.50, 1.00] # 0%, +50%, +100% do CTD_base
# A margem efetiva será 30%, 45%, 60%
# O valor aqui é o multiplicador da margem base. (1+0) = 30%, (1+0.5)=45% etc.
# Correto é aplicar a margem diretamente
margens_venda_efetivas_perc = [0.30, 0.45, 0.60]
variacoes_ganho_cliente_perc_sobre_base = [0, -0.20, -0.50] # 0%, -20%, -50% do GBAC_base

# --- 3. Geração dos Dados dos Cenários ---
dados_cenarios = []
cenario_id = 0

for var_ctd_fator in variacoes_ctd_perc_sobre_base:
    ctd_cenario = ctd_base * (1 + var_ctd_fator)
    
    for margem_venda_efetiva in margens_venda_efetivas_perc:
        receita_dev_cenario = ctd_cenario * (1 + margem_venda_efetiva)
        lucro_dev_cenario = receita_dev_cenario - ctd_cenario
        
        for var_ganho_cliente_fator in variacoes_ganho_cliente_perc_sobre_base:
            cenario_id += 1
            
            ganho_bruto_cliente_anual_cenario = ganho_bruto_cliente_anual_base * (1 + var_ganho_cliente_fator)
            
            bonus_calculado_anual_cenario = ganho_bruto_cliente_anual_cenario * percentual_bonus_consultoria
            receita_bonus_anual_efetiva_cenario = min(bonus_calculado_anual_cenario, teto_bonus_anual_consultoria)
            
            lucro_bonus_total_cenario_5a = (receita_bonus_anual_efetiva_cenario - bonus_custo_gestao_anual) * 5
            
            lucro_total_consultoria_cenario_5a = lucro_dev_cenario + lucro_bonus_total_cenario_5a
            
            gasto_cliente_total_cenario_5a = receita_dev_cenario + (receita_bonus_anual_efetiva_cenario * 5)
            
            ganho_liquido_cliente_cenario_5a = (ganho_bruto_cliente_anual_cenario * 5) - gasto_cliente_total_cenario_5a
            
            dados_cenarios.append({
                'Cenário ID': cenario_id,
                'Variação CTD (%)': f"{var_ctd_fator*100:.0f}%",
                'Margem Venda Efetiva (%)': f"{margem_venda_efetiva*100:.0f}%",
                'Variação Ganho Cliente Anual (%)': f"{var_ganho_cliente_fator*100:.0f}%", # Mudou de "Variação Bônus"
                'CTD Cenário (R$)': round(ctd_cenario, 2),
                'Receita Dev. Cenário (R$)': round(receita_dev_cenario, 2),
                'Lucro Dev. Cenário (R$)': round(lucro_dev_cenario, 2),
                'GBAC Cenário (R$)': round(ganho_bruto_cliente_anual_cenario, 2), # Adicionado
                'RBAC Cenário (R$)': round(receita_bonus_anual_efetiva_cenario, 2), # Mudou de "Receita Bônus Anual Cenário"
                'Lucro Bônus Total (5a) (R$)': round(lucro_bonus_total_cenario_5a, 2),
                'Lucro Total Consultoria (5a) (R$)': round(lucro_total_consultoria_cenario_5a, 2),
                'Gasto Cliente Total (5a) (R$)': round(gasto_cliente_total_cenario_5a, 2),
                'Ganho Líquido Cliente (5a) (R$)': round(ganho_liquido_cliente_cenario_5a, 2)
            })

df_cenarios = pd.DataFrame(dados_cenarios)

# --- 4. Visualizações Interativas com Plotly ---

# Mostrar a tabela de resultados no console para verificação
# print("Tabela de Resultados dos Cenários:")
# print(df_cenarios.to_string())
# print("\n")


# Gráfico 1: Lucro da Consultoria vs. Ganho Líquido do Cliente
fig1 = px.scatter(df_cenarios, 
                  x='Ganho Líquido Cliente (5a) (R$)', 
                  y='Lucro Total Consultoria (5a) (R$)',
                  color='Variação CTD (%)',
                  size_max=15, # Ajustar tamanho para melhor visualização
                  symbol='Variação Ganho Cliente Anual (%)', # Mudou
                  hover_name='Cenário ID',
                  hover_data=df_cenarios.columns, # Mostrar todos os dados no hover
                  title='Elasticidade: Lucro Consultoria vs. Ganho Líquido Cliente (5 Anos)')
fig1.update_layout(xaxis_title="Ganho Líquido do Cliente (5 Anos)",
                   yaxis_title="Lucro Total da Consultoria (5 Anos)",
                   legend_title_text='Legenda')
fig1.show()

# Gráfico 2: Impacto das variações no Lucro Total da Consultoria (Bar Chart)
fig2 = px.bar(df_cenarios.sort_values('Lucro Total Consultoria (5a) (R$)'), 
              x='Cenário ID', 
              y='Lucro Total Consultoria (5a) (R$)',
              color='Variação Ganho Cliente Anual (%)', # Mudou
              facet_col='Variação CTD (%)',
              facet_row='Margem Venda Efetiva (%)',
              hover_name='Cenário ID',
              hover_data=['GBAC Cenário (R$)', 'RBAC Cenário (R$)'],
              title='Lucro Total da Consultoria por Cenário')
fig2.update_xaxes(type='category') 
fig2.show()

# Gráfico 3: Comparativo Lucro Consultoria e Ganho Cliente por Cenário
df_melted = df_cenarios.melt(id_vars=['Cenário ID', 'Variação CTD (%)', 'Margem Venda Efetiva (%)', 'Variação Ganho Cliente Anual (%)'], 
                             value_vars=['Lucro Total Consultoria (5a) (R$)', 'Ganho Líquido Cliente (5a) (R$)'],
                             var_name='Métrica Financeira', 
                             value_name='Valor (R$)')

fig3 = px.bar(df_melted, 
              x='Cenário ID', 
              y='Valor (R$)',
              color='Métrica Financeira',
              barmode='group',
              hover_name='Cenário ID',
              hover_data=['Variação CTD (%)', 'Margem Venda Efetiva (%)', 'Variação Ganho Cliente Anual (%)'],
              title='Comparativo: Lucro Consultoria vs. Ganho Líquido Cliente')
fig3.update_xaxes(type='category')
fig3.show()

# Gráfico 4: Parallel Coordinates Plot para visualizar trade-offs
df_parallel = df_cenarios.copy()

# Converter colunas de variação para numérico para o plot
df_parallel['CTD_Var_Num'] = df_parallel['Variação CTD (%)'].str.rstrip('%').astype(float) / 100
df_parallel['Margem_Num'] = df_parallel['Margem Venda Efetiva (%)'].str.rstrip('%').astype(float) / 100
df_parallel['Ganho_Cliente_Var_Num'] = df_parallel['Variação Ganho Cliente Anual (%)'].str.rstrip('%').astype(float) / 100 # Mudou

dimensions = [
    dict(label='Variação CTD', values=df_parallel['CTD_Var_Num'],
         tickvals=[0, 0.5, 1], ticktext=['Base', '+50%', '+100%']),
    dict(label='Margem Venda', values=df_parallel['Margem_Num'],
         tickvals=[0.30, 0.45, 0.60], ticktext=['30%', '45%', '60%']),
    dict(label='Variação Ganho Cliente', values=df_parallel['Ganho_Cliente_Var_Num'], # Mudou
         tickvals=[0, -0.2, -0.5], ticktext=['Base', '-20%', '-50%']),
    dict(label='Lucro Consultoria (R$)', values=df_parallel['Lucro Total Consultoria (5a) (R$)']),
    dict(label='Ganho Cliente (R$)', values=df_parallel['Ganho Líquido Cliente (5a) (R$)'])
]

fig4 = go.Figure(data=go.Parcoords(
    line=dict(color=df_parallel['Lucro Total Consultoria (5a) (R$)'], 
              colorscale='viridis',
              showscale=True,
              cmin=df_parallel['Lucro Total Consultoria (5a) (R$)'].min(),
              cmax=df_parallel['Lucro Total Consultoria (5a) (R$)'].max()),
    dimensions=dimensions
))
fig4.update_layout(title='Análise de Trade-offs dos Cenários (Parallel Coordinates)')
fig4.show()